# Sparse Matrix Framework

Rather than looping through every cell and every timestep, we can instead just loop through every timestep, but solve the entire unstructured grid at once by setting up the grid as a sparse matrix. 

HEC-RAS 2D Sediment Transport solves an implicit Advection-Diffusion (transport) equation for the fractional total-load concentrations. The discretization produces a linear system of equations which may be represented by a sparse-matrix problem ([HEC-RAS 2D Sediment User Manual](https://www.hec.usace.army.mil/confluence/rasdocs/h2sd/ras2dsed/sediment-computation-options-and-tolerances/2d-computational-options)). The final form of the descritized total-load advection-diffusion equation can be seen [here](https://www.hec.usace.army.mil/confluence/rasdocs/d2sd/ras2dsedtr/numerical-methods/transport-equation). A similar approach can be taken for RAS2D-WQ. 

## Setup
Import packages and open results file. 

In [69]:
import xarray as xr 
import numpy as np
from scipy.sparse import *
from scipy.sparse.linalg import *

In [70]:
# Open Zarr
ds_zarr = xr.open_zarr('ugrid-example.zarr',
                       consolidated=True,  # http://xarray.pydata.org/en/stable/user-guide/io.html#consolidated-metadata
                      )

In [71]:
# additional required info - migrate to xarray setup code 
ds_zarr['volume'] = ds_zarr['depth'] * ds_zarr['faces_surface_area']
ds_zarr['edge_vertical_area'] = ds_zarr['depth'] * ds_zarr['edge_length']

In [72]:
ds_zarr

c:\Users\sjordan\Anaconda3\envs\pangeo\lib\site-packages\dask\array\core.py:1627: RuntimeWarning: overflow encountered in long_scalars
  cbytes = format_bytes(np.prod(self.chunksize) * self.dtype.itemsize)


<xarray.Dataset>
Dimensions:                 (nface: 5765, time: 289, nedge: 11164, 2: 2,
                             nmax_face: 7, node: 5774)
Coordinates:
    face_x                  (nface) float64 dask.array<chunksize=(5765,), meta=np.ndarray>
    face_y                  (nface) float64 dask.array<chunksize=(5765,), meta=np.ndarray>
    node_x                  (node) float64 dask.array<chunksize=(5774,), meta=np.ndarray>
    node_y                  (node) float64 dask.array<chunksize=(5774,), meta=np.ndarray>
  * time                    (time) datetime64[ns] 1900-01-02 ... 1900-01-03
Dimensions without coordinates: nface, nedge, 2, nmax_face, node
Data variables: (12/14)
    cells_surface_area      (nface) float32 dask.array<chunksize=(5765,), meta=np.ndarray>
    depth                   (time, nface) float32 dask.array<chunksize=(73, 1442), meta=np.ndarray>
    diffusion               (time, nedge) float64 dask.array<chunksize=(37, 2791), meta=np.ndarray>
    edge_face_connectivity  (nedge, 2) int32 dask.array<chunksize=(11164, 2), meta=np.ndarray>
    edge_length             (nedge) float32 dask.array<chunksize=(11164,), meta=np.ndarray>
    edge_nodes              (nedge, 2) int32 dask.array<chunksize=(11164, 2), meta=np.ndarray>
    ...                      ...
    face_to_face_dist       (nedge) float64 dask.array<chunksize=(11164,), meta=np.ndarray>
    face_velocity           (time, nedge) float32 dask.array<chunksize=(73, 2791), meta=np.ndarray>
    faces_surface_area      (nface) float32 dask.array<chunksize=(5765,), meta=np.ndarray>
    mesh2d                  int32 ...
    volume                  (time, nface) float32 dask.array<chunksize=(73, 1442), meta=np.ndarray>
    edge_vertical_area      (time, nface, nedge) float32 dask.array<chunksize=(73, 1442, 11164), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.8 UGRID-1.0 Deltares-0.10

In [64]:
# ultimately transition this to xarray attributes?
class Params:
    def __init__(self):
        self.diffusion_coefficient = 0.1

params = Params() 
    

## Sparse Matrix Setup
A sparse matrix is a matrix that is mostly zeroes. Here, we will set up a `NCELL x NCELL` sparse matrix. The diagonal values represent the cell, and then the non-zero values in that row/column represent other cells that share an edge with that cell. The MODFLOW-USG theoretical documentation contains an excellent example of an unstructured grid and the corresponding sparse matrix that can help illustrate this description ([USGS, 2013](https://pubs.usgs.gov/tm/06/a45/pdf/tm6-A45.pdf)) -- see Figures 18 & 19. 

Useful links:
* [Cell Hydraulic Properties](https://www.hec.usace.army.mil/confluence/rasdocs/d2sd/ras2dsedtr/numerical-methods/cell-hydraulic-properties)
* [Face Hydraulic Properties](https://www.hec.usace.army.mil/confluence/rasdocs/d2sd/ras2dsedtr/numerical-methods/face-hydraulic-properties)

In [ ]:
def sumDiff(ds):
    sum_conduct = np.zeros(len(ds['nface']))
    

        f1 = ds['edge_face_connectivity'].T[0]
        f2 = ds['edge_face_connectivity'].T[1]

    x1_coords = out['face_x'][f1]
    y1_coords = out['face_y'][f1]
    x2_coords = out['face_x'][f2]
    y2_coords = out['face_y'][f2]

    for i in range(len(connections.nodes)):
        node_1 = connections.nodes[i, 0]
        node_2 = connections.nodes[i, 1]
        sum_conduct[node_1] += connections.conduct[i]
        sum_conduct[node_2] += connections.conduct[i]
    return sum_conduct

The class below is a work in progress for defining coefficients of the LHS matrix. Upcoming items to address:
* Do I need to sum on diagonals?
* Advection term
* Leverage numpy operators  

In [65]:
class LHS:
    def __init__(self, ds, params):
        '''
        ds: xarray containing all geometry and ouptut results from RAS2D.
            Should follow UGRID conventions.
        params: A class instance containing additional parameters. 
            TBD if this will remain or if parameters will be integrated into xarray.
        t: timestep index  
        '''

        # initialize arrays that will define the sparse matrix 
        # rows, cols, data will all have a length of NCONNECTIONS * 2 + NCELL
        len_val = len(ds['nedge']) * 2 + len(ds['nface'])

        self.rows = np.zeros(len_val)
        self.cols = np.zeros(len_val)
        self.coef = np.zeros(len_val)
        
    def updateValues(self, ds, params, t):
        # diagonal terms
        # TO DO: do I need to loop here or can I just use array properties?
        # Do not need to loop: need to update code. 
        for i in ds['nface']:
            self.rows[i] = i
            self.cols[i] = i
            self.coef[i] = i # to update with information
            
            volume = ds['volume'][t][i]
            delta_t = ds[ds['time'][t+1] - ds['time'][t]] # will have to adjust for last timestep - or maybe directly save timestep?            

        # off-diagonal terms
        # TO DO: streamline w/o loop necessary (numpy array)
        for i in range(len(ds['nedge'])):
            # indices of connected faces for each edge 
            face_index_1 = ds['edge_face_connectivity'][i][0]
            face_index_2 = ds['edge_face_connectivity'][i][1]

            ### diffusion
            # required coefficients for diffusion
            # TO DO: should we be using depth or something else?
            edge_vertical_area = ds['edge_length'][i] * ds['depth'][face_index_1]
            distance = ds['face_to_face_dist'][i]
            # TO DO: will be negative for cell and negative for adjacent cell
            # How do we handle this?
            diffusion = edge_vertical_area * distance * params.diffusion_coefficient

            ### advection
            # TO DO: change face velocity to edge velocity for consistency?
            # depth?
            face_normal_water_flow = ds['face_velocity'][i] *ds['edge_length'][i] * ds['depth'][face_index_1]


            index_val = len(ds['nface']) + 2*i + 1
            self.rows[index_val] = face_index_1
            self.cols[index_val] = face_index_2
            self.coef[index_val] = i # need to calculate
            
            self.rows[index_val + 1] = face_index_2
            self.cols[index_val + 1] = face_index_1
            self.coef[index_val + 1] = i # need to calculate